In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class LipReadingModel(nn.Module):
    def __init__(self, dim_input_size, twoD_input_size, hidden_size, num_classes):
        super(LipReadingModel, self).__init__()

        self.lstm_dim = nn.LSTM(dim_input_size, hidden_size, batch_first=True)
        self.lstm_2d_1 = nn.LSTM(twoD_input_size, hidden_size, batch_first=True)
        self.lstm_2d_2 = nn.LSTM(twoD_input_size, hidden_size, batch_first=True)

        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 3, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x_dim, x_2d_1, x_2d_2):
        # LSTM expects (batch, sequence, feature)
        _, (h_dim, _) = self.lstm_dim(x_dim)
        _, (h_2d_1, _) = self.lstm_2d_1(x_2d_1)
        _, (h_2d_2, _) = self.lstm_2d_2(x_2d_2)

        # Use only the final hidden state
        h_cat = torch.cat((h_dim[-1], h_2d_1[-1], h_2d_2[-1]), dim=1)
        return self.fc(h_cat)
    
    

In [ ]:
model = LipReadingModel(
    dim_input_size = 74,
    twoD_input_size = 148, # Assuming 74 landmarks with 2D coordinates (x, y) and velocities
    hidden_size = 148,  # Hidden size for LSTM layers (can be adjusted)
    num_classes = NUM_CLASSES  # TODO Define the number of classes for your task (words, phonemes, etc.)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()

    for x_dim_batch, x_2d_1_batch, x_2d_2_batch, y_batch in data_loader:        
        outputs = model(x_dim_batch, x_2d_1_batch, x_2d_2_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')
